# Aim
Check whether DECD param give consistent fidelities with Qutip Simulations

# Imports

In [8]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')


In [9]:
# for param file 
#%%
# note: timestamp can't use "/" character for h5 saving.
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"
END_OPT_STRING = "\n" + "=" * 60 + "\n"
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time

In [10]:
# for pulse file
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks
from scipy.optimize import fmin



# Finding Param

In [11]:
from DECD_param import FixedBatchOptimizer as BO

In [12]:
#The initial oscillator state.
N1 =5
N2 =5
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #initial state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

#Target
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

#Optimization of ECD Circuit parameters (betas, gammas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 4, #circuit depth
'N_multistart' : 1, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
#"initial_params": init_params,
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



In [13]:
opt = BO(**opt_params)
opt.print_info()

None
Fixed construct needed matrices called
optimization_type: state transfer
N_multistart: 1
N_blocks: 4
term_fid: 0.999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 5
N_cav2: 5
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [ 0.48017-1.87195j  0.06786+2.35703j -0.74971+1.03073j -0.12688-1.65328j]
gammas:         [-2.66547-1.36016j  1.0707 +1.25087j -0.74816-0.32879j  0.30554-2.16833j]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [   0.      -126.9297   106.60694  120.28172]
etas (deg):    [89.99999 89.99999 89.99999 89.99999]
thetas (deg):  [-112.04588  112.25204  148.73178  -87.48447]
Max Fidelity:  0.000102




In [14]:
opt.optimize()

Start time: 2022-05-23 08:21:38
 Epoch: 9 / 100 Max Fid: 0.993431 Avg Fid: 0.993431 Max dFid: -0.004864 Avg dFid: -0.004864 Elapsed time: 0:00:07.005136 Remaing time: 0:01:10.829713
 max dFid: -0.004864
dFid stop: 0.000001


 Optimization stopped.  No dfid is greater than dfid_stop

optimization_type: state transfer
N_multistart: 1
N_blocks: 4
term_fid: 0.999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 5
N_cav2: 5
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [-1.39062-3.38146j -1.5806 +1.74659j -2.60015+0.6983j  -0.5705 -1.70699j]
gammas:         [-0.59622-1.88044j  0.33705+0.70574j  0.21859+0.1574j  -0.0442 -0.97482j]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [   0.      -135.86058   75.96234  10

'2022-05-23 08:21:38'

In [16]:
# saving angles 
betas = opt.best_circuit()['betas']
gammas = opt.best_circuit()['gammas']
phis = opt.best_circuit()['phis']
thetas = opt.best_circuit()['thetas']
etas = opt.best_circuit()['etas']
params = [np.real(betas), np.imag(betas), np.real(gammas), np.imag(gammas), phis, thetas, etas]
for i in range(len(params)):
    params[i] = [float(k) for k in params[i]]

fname = 'Test_00_10_May23.txt'
a_file = open(fname, "w")
np.savetxt(a_file, params)
a_file.close()

# Verifying with ECD single mode

In [18]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram\\ECD_control')


In [19]:
from ECD_control import ECD_optimization
from ECD_control import ECD_pulse_construction

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.batch_optimizer import BatchOptimizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5229363450648263752
 xla_global_id: -1]

In [20]:
#The target oscillator state.
N =5
Fock = 1
psi_t = qt.basis(N,Fock) #target state

In [21]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 4, #circuit depth
'N_multistart' : 1, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 200, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.995, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,1), psi_t)], #end in |e> |target>.
'name' : 'Fock %d' % Fock, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [22]:
ecd_opt = BatchOptimizer(**opt_params)

#print optimization info. 
ecd_opt.print_info()

optimization_type: state transfer
N_multistart: 1
N_blocks: 4
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 200
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
N_cav: 5
filename: Fock 1.h5

Best circuit parameters found:
betas:         [ 0.86667+1.25915j  1.53886+0.64929j  0.28542-0.06872j -1.24618+1.60091j]
alphas:        [0.+0.j]
phis (deg):    [   0.      -177.40057   80.07895  175.69635]
thetas (deg):  [ -3.95076 -64.84867 -75.5252  104.75389]
Max Fidelity:  0.013849




In [24]:
decd_opt = opt
ecd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.phis,
                decd_opt.thetas,)

<tf.Tensor: shape=(), dtype=float32, numpy=3.8809518e-05>

In [25]:
decd_opt.betas_rho

<tf.Variable 'betas_rho:0' shape=(4, 1) dtype=float32, numpy=
array([[3.656244 ],
       [2.3556032],
       [2.6922877],
       [1.7997965]], dtype=float32)>

In [26]:
decd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.gammas_rho,
                decd_opt.gammas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.alphas2_rho,
                decd_opt.alphas2_angle,
                decd_opt.phis,
                decd_opt.etas,
                decd_opt.thetas,)

<tf.Tensor: shape=(), dtype=float32, numpy=0.99341184>